In [6]:
import json
import pandas as pd
import folium
import googlemaps
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import rc 
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import requests

In [20]:
state_data = pd.read_csv("./data/02. US_Unemployment_Oct2012.csv", encoding= "utf-8")
state_data.tail(2)

,State,Unemployment
48,WI,6.8
49,WY,5.1


In [8]:
m = folium.Map([43, -102], zoom_start=3)

folium.Choropleth(
    geo_data="./data/02. us-states.json", # 경계선 좌표값이 담긴 데이터
    data=state_data, # Series or DataFrame 
    columns=["State", "Unemployment"], # DataFrame columns 
    key_on="feature.id",
    fill_color="BuPu",               #BuPu
    fill_opacity=0.5, # 0~1 
    line_opacity=0.2, # 0~1
    legend_name="Unemployment rate (%)"    
).add_to(m)

m

In [9]:
df = pd.read_csv("./data/02. 서울특별시 동작구_주택유형별 위치 정보 및 세대수 현황_20210825.csv", encoding="cp949")
df.tail(2)

,연번,분류,건물명,행정동,주소,세대수,위도,경도
165,166,연립주택,능내연립,사당5동,서울특별시 동작구 사당로8길 39,22,37.483599,126.968672
166,167,연립주택,천록,대방동,서울특별시 동작구 등용로 43,29,37.505475,126.933434


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      167 non-null    int64  
 1   분류      167 non-null    object 
 2   건물명     167 non-null    object 
 3   행정동     167 non-null    object 
 4   주소      167 non-null    object 
 5   세대수     167 non-null    int64  
 6   위도      163 non-null    float64
 7   경도      163 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 10.6+ KB


In [11]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 166
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      163 non-null    int64  
 1   분류      163 non-null    object 
 2   건물명     163 non-null    object 
 3   행정동     163 non-null    object 
 4   주소      163 non-null    object 
 5   세대수     163 non-null    int64  
 6   위도      163 non-null    float64
 7   경도      163 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 11.5+ KB


In [12]:
df = df.reset_index(drop=True)
df.tail(2)

,연번,분류,건물명,행정동,주소,세대수,위도,경도
161,166,연립주택,능내연립,사당5동,서울특별시 동작구 사당로8길 39,22,37.483599,126.968672
162,167,연립주택,천록,대방동,서울특별시 동작구 등용로 43,29,37.505475,126.933434


In [14]:
#del df["연번"]

In [15]:
df.columns

Index(['연번 ', '분류 ', '건물명', '행정동', '주소', '세대수', '위도', '경도'], dtype='object')

In [16]:
df = df.rename(columns={"연번 ": "연번", "분류 ": "분류"})
df.연번[:10]

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: 연번, dtype: int64

In [17]:
df.tail(2)

,연번,분류,건물명,행정동,주소,세대수,위도,경도
161,166,연립주택,능내연립,사당5동,서울특별시 동작구 사당로8길 39,22,37.483599,126.968672
162,167,연립주택,천록,대방동,서울특별시 동작구 등용로 43,29,37.505475,126.933434


In [18]:
df.describe()

,연번,세대수,위도,경도
count,163.000000,163.000000,163.000000,163.000000
mean,84.153374,371.920245,37.497442,126.949817
std,48.016276,413.115354,0.009532,0.019861
min,1.000000,21.000000,37.477376,126.906940
25%,43.500000,86.000000,37.490626,126.933284
50%,84.000000,199.000000,37.496940,126.949902
75%,125.500000,518.500000,37.505321,126.967196
max,167.000000,2621.000000,37.514280,126.981966


In [19]:
# folium 

m = folium.Map(location=[37.50589466533131, 126.93450729567374], zoom_start=13)

for idx, rows in df.iterrows():
    
    # location 
    lat, lng = rows.위도, rows.경도
    
    # Marker 
    folium.Marker(
        location=[lat, lng],
        popup=rows.주소,
        tooltip=rows.분류, 
        icon=folium.Icon(
            icon="home",
            color="lightred" if rows.세대수 >= 199 else "lightblue",
            icon_color="darkred" if rows.세대수 >= 199 else "darkblue",
        )
    ).add_to(m)

    # CircleMarker
    folium.Circle(
        location=[lat, lng],
        radius=rows.세대수 * 0.5, 
        fill=True, 
        color="pink" if rows.세대수 >= 518 else "green",
        fill_color="pink" if rows.세대수 >= 518 else "green",
    ).add_to(m)
    
m   